<a href="https://colab.research.google.com/github/slng09/advance-task22/blob/main/schoolcleanfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#--- suppress warnings ---
import warnings
warnings.filterwarnings("ignore")

In [31]:
# Read CSV
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded["SouthAustraliasch.csv"]),sep=',')
df.head()

Saving SouthAustraliasch.csv to SouthAustraliasch (3).csv


,Number,Name,Address,Suburb,Post Code,Latitude,Longitude,Category,Subtype,Subtype Name,Department
0,102,East Murray Area School,1182 East Murray Area School Road,Mindarie,5309,-34.815955,140.216073,SCHL,AREA,Area Schools,DEPT
1,104,Coromandel Valley Primary School,339 Main Road,Coromandel Valley,5051,-35.033562,138.626758,SCHL,PS,Primary Schools,DEPT
2,105,Glossop High School Senior Campus,50 Burgess Drive,Berri,5343,-34.279402,140.612880,SCHL,HIGH,High/Secondary Schools,DEPT
3,106,Cowandilla Primary School,21 Jenkins Street,Cowandilla,5033,-34.934543,138.558874,SCHL,PS,Primary Schools,DEPT
4,107,Crafers Primary School,55 Piccadilly Road,Crafers,5152,-34.993387,138.713480,SCHL,PS,Primary Schools,DEPT


In [32]:
df.isnull().sum()

Number           0
Name             0
Address         34
Suburb           0
Post Code        0
Latitude         0
Longitude        0
Category         0
Subtype          0
Subtype Name     0
Department       0
dtype: int64

In [ ]:
def clean_column_name(column_names):
    clean_column_names = []
    for c in column_names:
        c = c.lower().replace(' ','_')
        clean_column_names.append(c)
    return clean_column_names    

df.columns = clean_column_name(df.columns)

In [ ]:
#change column names
df.columns = ['stop_id', 'stop_name', 'latitude', 'longtitude', 'mode', 'state_code'] 
df.columns

Index(['stop_id', 'stop_name', 'latitude', 'longtitude', 'mode', 'state_code'], dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2261 entries, 0 to 2260
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Number        2261 non-null   int64  
 1   Name          2261 non-null   object 
 2   Address       2227 non-null   object 
 3   Suburb        2261 non-null   object 
 4   Post Code     2261 non-null   int64  
 5   Latitude      2261 non-null   float64
 6   Longitude     2261 non-null   float64
 7   Category      2261 non-null   object 
 8   Subtype       2261 non-null   object 
 9   Subtype Name  2261 non-null   object 
 10  Department    2261 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 194.4+ KB


In [33]:
duplicate = df[df.duplicated()]
print("Duplicate Rows :")
duplicate

Duplicate Rows :


,Number,Name,Address,Suburb,Post Code,Latitude,Longitude,Category,Subtype,Subtype Name,Department


In [ ]:
#--- take the means to replace the zero 

df['state'] = df['state'].replace([np.nan], ['Brisbane'])

In [ ]:
address_list = df['Address'].to_list() 
latitude = df['Latitude'].to_list()
longitude = df['Longitude'].to_list()

geolocator = Nominatim(user_agent="test")

def reverse_geocoding(lat, lon):
    try:
        location = geolocator.reverse(f"{lat}, {lon}")
        return location.raw['display_name']
    except:
        return None
    
# Find rows in df with a missing address
df_nan = df[df.Address.isnull()]

# Reverse geocode the latitude and longitude to get the missing addresses
addresses = np.vectorize(reverse_geocoding)(df_nan['Latitude'], df_nan['Longitude'])

# Update the Address column in df_nan with the reverse geocoded addresses
df_nan['Address'] = addresses

# Update df with the modified df_nan
df.update(df_nan)

df_nan

In [ ]:
name_list = df['Name'].to_list() 
locator = Nominatim(user_agent="my-application", timeout=20)

def get_address(name):
    """Get the address of a location using its name"""
    try:
        location = locator.geocode(f"{name}")
        return location.raw['display_name']
    except:
        return None
        
# Find rows in df with a missing address
df_nan = df[df.Address.isnull()]

# Get the missing addresses
addresses = df_nan['Name'].apply(get_address)

# Update the Address column in df_nan with the missing addresses
df_nan['Address'] = addresses

# Update df with the modified df_nan
df.update(df_nan)

df_nan


In [28]:
df.isnull().sum()

Number          0
Name            0
Address         0
Suburb          0
Post Code       0
Latitude        0
Longitude       0
Category        0
Subtype         0
Subtype Name    0
Department      0
dtype: int64

In [ ]:
# Replace NaN with zero on all columns 
df = df.fillna(0)

In [29]:
from google.colab import files
df.to_csv('SouthAustraliasch.csv') 
files.download('SouthAustraliasch.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>